# Filtered Signals
### Effect of Filtering on Pitch & Timbre

Author: Dirk Van Compernolle   
Created: 05/10/2021   
Last Modification:

#### LIMITATIONS in Google Colab !! 
+ The Autoplay function does not always work properly on Ipython < 6.0 [Google Colab] .  Just hit the play button to force audio output.  
+ Moreover in these versions of IPython all sounds are played with amplitude normalization (over the full signal) and hence the amplitude setting will have no effect on the loudness of the played sound

## Do all the imports and define some helper functions
You need to run the next 2 cells first before starting the demos   
After that you can run the different demos in any order

In [2]:
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive, interactive_output
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy import signal
from IPython.display import display, clear_output, Audio, HTML
import time, copy, math
import librosa
from scipy.io import wavfile


try:
  import google.colab
  IN_COLAB = True 
except:
  IN_COLAB = False
### REQUIRED PACKAGE: pyspch !! ###
try:
  import pyspch
except:
  ! pip install git+https://github.com/compi1234/pyspch.git

import pyspch.spg as Sps
import pyspch.display as Spd
import pyspch.audio as Spa
from pyspch.constants import EPS_FLOAT

#
# synthesize a set of harmonics with given frequencies, amplitude and phases
def synth_harmonics(freqs = [200.], amps = [1.], phases = None, dur = 0.5,sample_rate=8000):
    N = len(freqs)
    if  len(amps) != N: amps = [amps[0]] * N
    if phases is None: phases = [0.]*N
    elif len(phases) != N : [phases[0]]*N        
    t = np.linspace(0.0, dur, int(dur*sample_rate), endpoint=False)
    y = 0*t
    for k in range(N):
        y = y+ amps[k]*np.sin( 2.*np.pi*freqs[k]*t + phases[k])
    return y,t

# set of synthesis methods with one base frequency
def synth(sigtype='sin', freq=200.0, amp=1.0, phi=0.0, sample_rate=8000, dur=0.25):
    t = np.linspace(0.0, dur, int(dur*sample_rate), endpoint=False)
    npts = len(t)
    tt = 2.*np.pi*(freq*t) + phi
    if sigtype == 'sin':
        y = np.sin(tt)
    elif sigtype == 'square':
        y = signal.square(tt)
    elif sigtype == 'sawtooth':
        y = signal.sawtooth(tt)
    elif sigtype == 'triangle':
        y = signal.sawtooth(tt,width=.5)
    elif sigtype == 'pulsetrain':
        y = np.zeros(t.shape)
        nperiod = int((1./freq)*sample_rate)
        for k in range(5,npts,nperiod):
            y[k] = 1.
    elif sigtype == 'chirp 1:20':
        y = signal.chirp(t,freq,dur,20.*freq,method='linear')
    elif sigtype == 'chirp 20:1':
        y = signal.chirp(t,20*freq,dur,freq,method='linear')
    elif sigtype == 'gausspulse':
        y = signal.gausspulse(t-dur/2.,fc=freq,bw=.4)
    elif sigtype == 'white noise':
        y = np.random.randn(len(t))/4.
    elif sigtype == 'modulated white noise':
        y = np.random.randn(len(t))*(np.sin(tt)+1.)/8.
    elif sigtype == 'Dual Tone':   #DTMF tone ratios is approximately 21/19 ~ 1.1054 (697,770,852,941)*(1209,1336,1477)
                              # row/col ratio is 1.735
        tt1 = 2.*np.pi*(1.735*freq*t) + phi
        y = 0.5*np.sin(tt) + 0.5*np.sin(tt1) 
    else:
        filename = "https://homes.esat.kuleuven.be/~spchlab/data/misc/"+sigtype+".wav"
        y, sample_rate = Spa.load(filename)
        amp = 1.0
        t = np.linspace(0.0, len(y)/float(sample_rate), len(y), endpoint=False)
    return amp*y, t, sample_rate

# apply ADSR style envelope (attack, decay, systain, release)
def adsr_envelope(y,instr=None,sample_rate=8000,s=1., A=0., D=0., R=0.):
    adsr={
        'piano':{'s':1.,'A':.005,'D':0.,'R':.025},
        'guitar':{'s':1.,'A':.015,'D':0.,'R':.015},
        'trumpet':{'s':.32,'A':.030,'D':0.110,'R':.100},
        'cello':{'s':.47,'A':.040,'D':0.290,'R':.200}
    }
    if instr is not None:
        s=adsr[instr]['s']
        A=adsr[instr]['A']
        D=adsr[instr]['D']
        R=adsr[instr]['R']
    
    y1= np.zeros(y.shape)
    n4 = len(y)
    t4 = n4/sample_rate
    t1 = A
    t2 = A+D
    t3 = t4 - R
    n1 = int(t1*sample_rate)
    n2 = int(t2*sample_rate)
    n3 = int(t3*sample_rate)
    y1[0:n1] = y[0:n1]*(.5+.5*signal.sawtooth(2*np.pi*np.linspace(0,1,n1,endpoint=False)))
    y1[n1:n2] = y[n1:n2]*(.5+.5*s-.5*(1-s)*signal.sawtooth(2*np.pi*np.linspace(0,1,n2-n1,endpoint=False)))
    y1[n2:n3] = y[n2:n3]*s
    y1[n3:n4] = y[n3:n4]*s* np.exp(-np.log(100)*np.linspace(0,1,n4-n3,endpoint=False))
    return(y1)


# The following code will increase the default width of your Jupyter notebook cells
# Supposed to work well 
display(HTML(data="""
<style>
    div#notebook-container    { width: 99%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

# a default boxed layout
def box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

In [3]:
signal_types = [ 'friendly','expansionist','timit1','timit2','splat','train','sin', 'square', 'triangle', 'sawtooth', 'pulsetrain','chirp 1:20','chirp 20:1']
display_types = ['spectrum','spectrogram','melspectrogram']
filter_types = ['bandpass','bandstop']
class Filtered_Signals(widgets.VBox):
    def __init__(self,dur=.5,sample_rate=8000,f0=200.,freq_range=[300.,3400.],figsize=(10,6),dpi=100):
        super().__init__()
        
        if sample_rate < 4000.:
            print("WARNING: sampling rate should not be lower than 4000.0 Hz and has been reset.")
            sample_rate = 4000.0

        self.sample_rate = sample_rate
        self.dur = dur     
        
        self.disptype = 'spectrogram'
        self.sigtype = 'friendly'
        self.filtertype = 'bandpass'
        self.f0 = f0
        self.freq = freq_range
        self.amp = 1.

        # create the widgets
        self.wg_disptype = widgets.Dropdown(options=display_types,value=self.disptype,description="Display")
        self.wg_sigtype = widgets.Dropdown(options=signal_types,value=self.sigtype,description="Signal") 
        self.wg_filtertype = widgets.Dropdown(options=filter_types,value=self.filtertype,description="Filter")
        self.wg_freq = widgets.FloatRangeSlider(value=self.freq,step=100.,min=100,max=self.sample_rate/2-100,description='Freq Range (Hz)',
                                                layout=widgets.Layout(width='70%'),continous_update=False)

        self.wg_clear_log = widgets.Button(description='Clear the log')
        self.wg_clear_log.layout.width='50%'
        
        # link to the widget observers

        self.wg_sigtype.observe(self.sigtype_observe,'value')  
        self.wg_disptype.observe(self.disptype_observe,'value') 
        self.wg_filtertype.observe(self.filtertype_observe,'value') 
        self.wg_freq.observe(self.freq_observe,'value')


        # setup the outputs 
        self.audio1 = widgets.Output()
        self.audio2 = widgets.Output()
        self.logscr = widgets.Output()
        self.out1 = widgets.Output(layout=box_layout())
        self.out2 = widgets.Output(layout=box_layout())
        self.UI = widgets.VBox( [self.wg_disptype,  self.wg_sigtype,  self.wg_filtertype, self.wg_freq],
                                 layout=box_layout())
        
        
        self.left_screen = widgets.VBox([self.out1, self.audio1],layout=box_layout())
        self.right_screen = widgets.VBox([self.out2, self.audio2],layout=box_layout())
                                          
        self.fig1 = Spd.SpchFig(row_heights=[1.,1.], figsize=figsize,dpi=dpi)
        self.fig2 = Spd.SpchFig(row_heights=[1.,1.], figsize=figsize,dpi=dpi)
        self.update()
        plt.close()          # avoids output of dummy figure on startup

        # attach children to the VBox class
        self.children = [ self.UI, widgets.HBox([self.left_screen,self.right_screen ]) ] 


    def update(self):
        y,x,self.sample_rate = synth(sigtype=self.sigtype,freq=self.f0,sample_rate=self.sample_rate,dur=self.dur)
        self.wg_freq.max = self.sample_rate/2-100.
        sos = signal.butter(20, self.freq, self.filtertype, fs=self.sample_rate, output='sos')
        y2 = signal.sosfilt(sos, y)
    
        ax = self.fig1.axes
        ax[0].cla()
        ax[1].cla()
        ax = self.fig2.axes
        ax[0].cla()
        ax[1].cla()
        
        if self.disptype == 'spectrum':
            freq_ax,spec_pow = signal.periodogram(y,fs=self.sample_rate,scaling='spectrum')
            # note: the 2.*spec is necessary to incorporate the duplicate information in negative frequencies 
            spec = 10.0*np.log10(2.*spec_pow+EPS_FLOAT)
            ylabel = 'Log Spectrum (dB)'
            yrange = [-80, 2]

            self.fig1.add_line_plot(y,iax=0,x=x,yrange=[-1,1.],xlabel='Time(sec)',title='Waveform')
            self.fig1.add_line_plot(spec,iax=1,x=freq_ax,yrange=yrange,xlabel='Frequency (Hz)',ylabel=ylabel,title='Spectrum',grid=True)
        
            freq_ax,spec2 = signal.periodogram(y2,fs=self.sample_rate,scaling='spectrum')
            spec2 = 10.0*np.log10(2.*spec2+EPS_FLOAT)
            self.fig2.add_line_plot(y2,iax=0,x=x,yrange=[-1,1.],xlabel='Time(sec)',title='Waveform')
            self.fig2.add_line_plot(spec2,iax=1,x=freq_ax,yrange=yrange,xlabel='Frequency (Hz)',ylabel=ylabel,title='Spectrum',grid=True)            
            
        else:
            if self.disptype == 'spectrogram':
                spg1 = Sps.spectrogram(y,sample_rate=self.sample_rate,preemp=0.)
                spg2 = Sps.spectrogram(y2,sample_rate=self.sample_rate,preemp=0.)
                ylabel="Frequency (Hz)"
                dy = None

            elif self.disptype == 'melspectrogram':
                spg1 = Sps.spectrogram(y,sample_rate=self.sample_rate,preemp=0.,n_mels=100)
                spg2 = Sps.spectrogram(y2,sample_rate=self.sample_rate,preemp=0.,n_mels=100)
                ylabel = "mel-channel"
                dy = 1;

            self.fig1 = Spd.PlotSpg(spgdata=spg1,wavdata=y,sample_rate=self.sample_rate,
                       ylabel=ylabel,title="Original Signal",dy=dy)                
            self.fig2 = Spd.PlotSpg(spgdata=spg2,wavdata=y2,sample_rate=self.sample_rate,dy=dy,
                    ylabel=ylabel,title="Filtered Signal")
            
        # here come the things that go to dedicated output widgets
        with self.out1:
            clear_output(wait=True)
            display(self.fig1)
        with self.out2:
            clear_output(wait=True)
            display(self.fig2)
        with self.audio1:
            clear_output(wait=True)
            try:
                display(Audio(data=y,rate=self.sample_rate,normalize=False,autoplay=False))
            except: 
                display(Audio(data=y,rate=self.sample_rate, autoplay=False))
        with self.audio2:
            clear_output(wait=True)
            try:
                display(Audio(data=y2,rate=self.sample_rate,normalize=False,autoplay=False))
            except: 
                display(Audio(data=y2,rate=self.sample_rate, autoplay=False))
                
     
    def disptype_observe(self,change):
        self.disptype = change.new
        self.update()
        

    def sigtype_observe(self,change):
        self.sigtype = change.new
        self.update()

    def filtertype_observe(self,change):
        self.filtertype = change.new
        self.update()

    def freq_observe(self,change):
        self.freq = change.new
        self.update()
   

## Filtered Speech

In this demo we let you experiment with bandpass and bandstop filters.  The digital filters we use are digital equivalents of common analog Butterworth filters.   

Note: This implies that a bandpass filter always has some leakage of signal in the range outside the frequency range that is passed and vice versa that a bandstop filter does not stop the frequencies for a 100% in the intended range.

In the left-hand side of the UI you find amplitude and spectrogram of the **unfiltered** signal in the right-hand side you find amplitude and spectrogram of the **filtered** signal.

### Exercise 1. Speech with Telephone Bandwidth

The classic analog telephone line has a bandwidth of 300-3400Hz approximately.
Choose any of the speech signals as input and observed the filtered signal (listen and look at the spectrogram).  What is the perceptual effect ?

Questions:   
- can you still identify all vowels
- which type of sounds are most heavily impact by this bandwidth reduction (be sure to listen to an input using a high sampling frequency as well, eg. misinterpret.wav)
- can you still determine the identity of the speaker ?
- is the presence of the fundamental frequency essential for speaker recognition, pitch recognition ?



### Exercise 2. Redundancy in bandpass-filtered Speech

The telephone bandwidth did not arise by chance or some analog filtering considerations.  It was the result of perceptual experiments at Bell Labs over 100 years ago and was deemed a good compromise between speech quality and needed bandwidth.

Let's now redo some of those experiments.

- 2.1 Bandpass filtering

    + In post WW-II era in several countries the telephone companies could not add telephone lines quickly enough to accomodate all new customers .  Therefore in the same appartment building often a single line with a 4kHz bandwidth was shared with 2 customers hence a 2kHz bandwidth telephone  was the best that could be offered.   Try to figure out what the best solution was: 
[0-2000Hz], [500-2500Hz], [1000-3000Hz] or [2000-4000Hz].

    + How does the visual effect of filtering change if you use a melspectrogram instead of the Fourier spectrogram ?

- 2.2 Bandstop filtering

It is acknowledge that F2 is of critical importance for vowel recognition.  Design a bandstop filter that eliminates most or all of the F2 frequency range.   Can speech still be understood and how do you explain this ?


### Exercise 3. Filtering of Harmonic Signals

Use some of the artificial signals for this demo and just look at the 'spectrum' view .  Redo som e of the experiments above to observe both filtering and influence on perception.

- Can you explain the source-filter model in this experiment ?
- Again observe the **missing fundamental** phenomenon in telephone speech

In [4]:
Filtered_Signals(dur=1,f0=100)

Filtered_Signals(children=(VBox(children=(Dropdown(description='Display', index=1, options=('spectrum', 'spect…